In [1]:
#Importing library

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import datetime

from scipy import stats
from scipy.stats import skew
from scipy.stats import norm
from scipy.stats.stats import pearsonr
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from math import sqrt


%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
#Importing dataset
train = pd.read_csv("dataset/train.csv")
val = pd.read_csv("dataset/train_val1.csv")

We prepare the target valuesy for all the sets

In [3]:
y=train['NumberOfSales']
y_val1=val['NumberOfSales']

We prepare the training input samples for all the sets

In [4]:
X=train.drop(['NumberOfSales', 'MeanMonthSales', 'Min_Sea_Level_PressurehPa'], axis=1)
X_val1=val.drop(['NumberOfSales', 'MeanMonthSales', 'Min_Sea_Level_PressurehPa'], axis=1)

In [5]:
res = {'number of trees' : [], 'number of attributes' : [], 'depth' : [], 'mse' : [], 'mae' : []}

In [7]:
for na in range(7,17):
    for nt in range(151,252,25):
        for depth in range(15,28,3):
                X=train.drop(['NumberOfSales'], axis=1)
                X_val1=val.drop(['NumberOfSales'], axis=1)
                forest = RandomForestRegressor(max_depth=depth, random_state=0, n_estimators=nt, max_features=na, n_jobs=-1)
                forest.fit(X, y)
                y_pred=forest.predict(X_val1)
                mse_val1=mean_squared_error(y_val1,y_pred)
                mae_val1=mean_absolute_error(y_val1,y_pred)
                res['number of trees'].append(nt)
                res['number of attributes'].append(na)
                res['depth'].append(depth)
                res['mse'].append(mse_val1)
                res['mae'].append(mae_val1)
                print(mse_val1, mae_val1, na, nt, depth)
                
r_df = pd.DataFrame(res)

0.0013019094620739695 0.02515148064844595 7 151 15
0.0012278115150219703 0.02412704843952567 7 151 18
0.0012082790252419787 0.023758169873889342 7 151 21
0.0012044357052523164 0.023673798142227333 7 151 24
0.0012141105364779608 0.023742381284825295 7 151 27
0.0013025786908151364 0.025140039650304816 7 176 15
0.001225865758160609 0.02408720923306677 7 176 18
0.0012086377356764556 0.02377568981930195 7 176 21
0.001205229712324393 0.02367609156157291 7 176 24
0.0012101116789124632 0.023713699024803806 7 176 27
0.0013028606604994896 0.025128882564433443 7 201 15
0.001224514325243343 0.024076537769360248 7 201 18
0.001206911850191988 0.02374279302189485 7 201 21
0.0012057498064359997 0.023673698490827202 7 201 24
0.0012100846333595217 0.023714110777461064 7 201 27
0.0013031023818481883 0.0251169714940656 7 226 15
0.0012273640705465889 0.024107648523291815 7 226 18
0.001207493897876076 0.02376343591715431 7 226 21
0.0012050903866584117 0.02367299361450957 7 226 24
0.0012056306871422093 0.023

0.001128148269577237 0.022944230611086275 13 176 24
0.0011238610671969855 0.022925428535771078 13 176 27
0.001124526989177385 0.023072688449692083 13 201 15
0.0011119608126727181 0.022856121311282333 13 201 18
0.001125564132949142 0.022921411670199985 13 201 21
0.001127159785537381 0.022936862922410612 13 201 24
0.0011174869007869773 0.022861109815759697 13 201 27
0.0011272288449764379 0.02308436999780048 13 226 15
0.0011100744201173252 0.022834363888987085 13 226 18
0.0011230025221060749 0.022901668591357154 13 226 21
0.0011230947665844273 0.02289961366129275 13 226 24
0.0011168801291504755 0.022851691266815 13 226 27
0.0011261041278944053 0.02308342160825491 13 251 15
0.001109430289532097 0.022830137335770113 13 251 18
0.0011229142403897897 0.022908759610191426 13 251 21
0.0011238854128652245 0.02289612411011194 13 251 24
0.001118471818398942 0.022872072770582997 13 251 27
0.0011358749869761152 0.023130559711428696 14 151 15
0.0011083527939096682 0.02282738385278126 14 151 18
0.00114

In [9]:
r_df.to_csv("parameters.csv")